In [ ]:
!rm -rf Dataset Trained_Model                   # Remove existing directory
!pip install -U gdown --pre >/dev/null          # Install gdown to download file from GDrive
!gdown 1u4WJLjYrbZHwdvFOHQXJqDTtco6F5hJ-        # Download dataset from GDrive by file ID
!unzip -q Dataset.zip; rm Dataset.zip           # Extract the dataset zip file

In [ ]:
import os
import cv2
import numpy as np
from random import sample
from skimage.io import imread
from sklearn.model_selection import train_test_split


IMAGES_PATH = 'Dataset/Images/'
MASKS_PATH  = 'Dataset/Masks/'
TEST_PATH   = 'Dataset/Test_Images/'

# Number of images to use (Larger the number, more RAM required)
N_IMAGES = 1500

# Imread each image and save to an array

sat_imgs = os.listdir(IMAGES_PATH)
msk_imgs = os.listdir(MASKS_PATH)
sat_imgs.sort(), msk_imgs.sort()

images = []
for image in sat_imgs[:N_IMAGES]:
    data = imread(IMAGES_PATH + image)
    images.append(data)

masks = []
for mask in msk_imgs[:N_IMAGES]:
    data = imread(MASKS_PATH + mask)
    data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    data = np.expand_dims(data, axis=-1)
    masks.append(data)

images = np.stack(images)
masks = np.stack(masks) / 255

train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.2, random_state=2)

pred_images = sample(os.listdir(IMAGES_PATH), 4)
pred_masks  = []
for mask_name in pred_images:
    mask_name = mask_name.replace('_sat.jpg', '_mask.png')
    pred_masks.append(mask_name)

del images, masks

print("Training Set")
print(train_images.shape)
print(train_masks.shape)
print("Testing Set")
print(test_images.shape)
print(test_masks.shape)

!rm -rf epochs Trained_Model; mkdir epochs

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
from tensorflow.keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
# Import Libraries
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, Activation, add
from tensorflow.keras.models import Model
import tensorflow as tf

def FCN8():
    img_input = Input(shape=(512, 512, 3))

# Block 1
    x = Conv2D(64, 3, activation='relu', padding='same') (img_input)
    x = Conv2D(64, 3, activation='relu', padding='same') (x)
    x = MaxPooling2D(2, strides=2) (x)
    x = Dropout(0.3) (x)

# Block 2
    x = Conv2D(128, 3, activation='relu', padding='same') (x)
    x = Conv2D(128, 3, activation='relu', padding='same') (x)
    x = MaxPooling2D(2, strides=2) (x)
    x = Dropout(0.5) (x)

# Block 3
    x = Conv2D(256, 3, activation='relu', padding='same') (x)
    x = Conv2D(256, 3, activation='relu', padding='same') (x)
    x = Conv2D(256, 3, activation='relu', padding='same') (x)
    x = MaxPooling2D(2, strides=2) (x)
    x = Dropout(0.5) (x)

    skip1 = Conv2D(256, 3, activation='relu', padding='same') (x)

# Block 4
    x = Conv2D(512, 3, activation='relu', padding='same') (x)
    x = Conv2D(512, 3, activation='relu', padding='same') (x)
    x = Conv2D(512, 3, activation='relu', padding='same') (x)
    x = MaxPooling2D(2, strides=2) (x)
    x = Dropout(0.5) (x)

    skip2 = Conv2D(256, 3, activation='relu', padding='same') (x)
    skip2 = Conv2DTranspose(256, kernel_size=2, strides=2) (skip2)

# Block 5
    x = Conv2D(512, 3, activation='relu', padding='same') (x)
    x = Conv2D(512, 3, activation='relu', padding='same') (x)
    x = Conv2D(512, 3, activation='relu', padding='same') (x)
    x = MaxPooling2D(2, strides=2) (x)
    x = Dropout(0.5) (x)


    x = Conv2D(4096 , (7, 7) , activation='relu' , padding='same') (x)
    x = Conv2D(4096 , (1, 1) , activation='relu' , padding='same') (x)

    x = Conv2D(256, 3, activation='relu', padding='same') (x)
    x = Conv2DTranspose(256, kernel_size=2, strides=2) (x)
    x = Conv2DTranspose(256, kernel_size=2, strides=2) (x)

    x = add([skip1, skip2, x])

    x = Conv2DTranspose(128, kernel_size=2, kernel_initializer='he_normal', strides=2) (x)
    x = Conv2DTranspose(32, kernel_size=2, kernel_initializer='he_normal', strides=2) (x)
    x = Conv2DTranspose(8, kernel_size=2, kernel_initializer='he_normal', strides=2) (x)

    x = Conv2D(1, 1, kernel_initializer='he_normal') (x)
    x = (Activation('sigmoid'))(x)

    model = Model(img_input, x)
    return(model)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from IPython.display import clear_output
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf

model_path = "./Final_Model/FCNN-8-Model.h5"
evals = []
scores = []


checkpointer = ModelCheckpoint(model_path, monitor="val_loss", mode="min", save_best_only = True, verbose=1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)

def model_status(epoch, accu, val_accu, loss, val_loss):
    [loss, accuracy, tp, fp, tn, fn, precision, recall] = model.evaluate(test_images, test_masks)
    f1_score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    evals.append(accuracy)
    scores.append(f1_score)

    f = plt.figure(figsize = (24, 16))
    gs = f.add_gridspec(4, 6)
    f.suptitle(f'Epoch: {epoch}', x=0.5, y=0.02)
    titles = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(0,4):
        data = [imread(f'Dataset/Images/{pred_images[i]}')]
        input_img = imread(f'Dataset/Images/{pred_images[i]}')
        mask_img = imread(f'Dataset/Masks/{pred_masks[i]}')
        test_data = np.asarray([input_img])
        output = model.predict(test_data, verbose=0)[0][:,:,0]
        data += [mask_img, output]
        for j in range(0,3):  
            f.add_subplot(gs[i, j])
            plt.imshow(data[j])
            plt.axis('off')
            if i == 0:
                plt.title(titles[j])

    ax = f.add_subplot(gs[0:2, 3:6])
    ax.tick_params(axis="y",direction="in", pad=-25)
    ax.text(.5,.95,'Model Accuracy', horizontalalignment='center', transform=ax.transAxes, weight='bold')
    plt.plot(accu)
    plt.plot(val_accu)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training Accuracy', 'Validation accuracy'], loc='lower right')
    
    ax = f.add_subplot(gs[2:4, 3:6])
    ax.tick_params(axis="y",direction="in", pad=-25)
    ax.text(.5,.95,'Model Loss', horizontalalignment='center', transform=ax.transAxes, weight='bold')
    plt.plot(loss)
    plt.plot(val_loss)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training loss', 'Validation loss'], loc='upper right')
    
    plt.savefig(f'epochs/{epoch}.png')
    plt.show()

class DisplayCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.accu = []
        self.val_accu = []
        self.loss = []
        self.val_loss = []
    def on_epoch_end(self, epoch, logs=None):
        self.accu.append(logs.get("accuracy"))
        self.val_accu.append(logs.get("val_accuracy"))
        self.loss.append(logs.get("loss"))
        self.val_loss.append(logs.get("val_loss"))
        clear_output(wait=True)
        model_status(epoch, self.accu, self.val_accu, self.loss, self.val_loss)

In [ ]:
with strategy.scope():
    model = FCN8()
    metrics = [
    'accuracy',
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall()]

In [ ]:
EPOCHS = 150
LEARNING_RATE = 0.0001
BATCH_SIZE = 56
adam = tf.keras.optimizers.Adam(LEARNING_RATE)

model.compile(optimizer=adam, loss=soft_dice_loss, metrics=metrics)

In [ ]:
history = model.fit(train_images,
                    train_masks,
                    validation_split = 0.2,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    callbacks = [checkpointer, DisplayCallback()])

In [ ]:
model.evaluate(test_images, test_masks)

In [ ]:
pred_images = sample(os.listdir(IMAGES_PATH), 4)
pred_masks  = []
for mask_name in pred_images:
    mask_name = mask_name.replace('_sat.jpg', '_mask.png')
    pred_masks.append(mask_name)

f = plt.figure(figsize = (15, 12), constrained_layout=True)
gs = f.add_gridspec(5, 6)
titles = ['Input Image', 'Ground Truth', 'Predicted Image']

for i in range(0,4):
    data = [imread(f'Dataset/Images/{pred_images[i]}')]
    input_img = imread(f'Dataset/Images/{pred_images[i]}')
    mask_img = imread(f'Dataset/Masks/{pred_masks[i]}')
    test_data = np.asarray([input_img])
    output = model.predict(test_data, verbose=0)[0][:,:,0]
    final_out = input_img
    for xi in range(len(output)):
        for yi in range(len(output[xi])):
            if output[xi][yi] > 0.1:
                final_out[xi][yi] = [255, 255, 0]
    data += [mask_img, final_out]
    for j in range(0,3):  
        f.add_subplot(gs[i, j])
        plt.imshow(data[j])
        plt.axis('off')
        if i == 0:
            plt.title(titles[j])
plt.show()